K-MEAN ALGORITHM

In [ ]:
import pandas
import numpy
from numpy.linalg import norm

import seaborn as sns
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
from matplotlib.image import imread

In [ ]:
datalist = pandas.read_excel('Absenteeism_at_work.xls')
datalist.head(10)

,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,Hit target,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,11,26,7,3,1,289,36,13,33,239554,97,0,1,2,1,0,1,90,172,30,4
1,36,0,7,3,1,118,13,18,50,239554,97,1,1,1,1,0,0,98,178,31,0
2,3,23,7,4,1,179,51,18,38,239554,97,0,1,0,1,0,0,89,170,31,2
3,7,7,7,5,1,279,5,14,39,239554,97,0,1,2,1,1,0,68,168,24,4
4,11,23,7,5,1,289,36,13,33,239554,97,0,1,2,1,0,1,90,172,30,2
5,3,23,7,6,1,179,51,18,38,239554,97,0,1,0,1,0,0,89,170,31,2
6,10,22,7,6,1,361,52,3,28,239554,97,0,1,1,1,0,4,80,172,27,8
7,20,23,7,6,1,260,50,11,36,239554,97,0,1,4,1,0,0,65,168,23,4
8,14,19,7,2,1,155,12,14,34,239554,97,0,1,2,1,0,0,95,196,25,40
9,1,22,7,2,1,235,11,14,37,239554,97,0,3,1,0,0,1,88,172,29,8


In [ ]:
Df = StandardScaler().fit_transform(datalist)

In [ ]:
def Centroid_initalize(data,random_state,n_clusters):
    numpy.random.RandomState(random_state)
    random_idx = numpy.random.permutation(data.shape[0])
    centroids = data[random_idx[:n_clusters]]
    return centroids

In [ ]:
def Centroid_calculation(data, Label,n_clusters):
    centroids = numpy.zeros((n_clusters, data.shape[1]))
    for l in range(n_clusters):
        centroids[l, :] = numpy.mean(data[Label == l, :], axis=0)
    return centroids

In [ ]:
def Distance_calculation(data, centroids,n_clusters):
    distance = numpy.zeros((data.shape[0], n_clusters))
    for l in range(n_clusters):
        row_norm = norm(data - centroids[l, :], axis=1)
        distance[:, l] = numpy.square(row_norm)
    return distance

In [ ]:
def Nearest_cluster(distance):
  #finding minimum distance
    return numpy.argmin(distance, axis=1)

In [ ]:
def Compute_sse(data, Label, centroids,n_clusters):
    distance = numpy.zeros(data.shape[0])
    for l in range(n_clusters):
        distance[Label == l] = norm(data[Label == l] - centroids[l], axis=1)
    return numpy.sum(numpy.square(distance))

In [ ]:
def fit(data,n_clusters, max_iter, random_state=420):
  centroids = Centroid_initalize(data,random_state,n_clusters)
  print(centroids.shape)
  for i in range(max_iter):
      centroid_previous = centroids
      distance = Distance_calculation(data, centroid_previous,n_clusters)
      Label = Nearest_cluster(distance)
      centroids = Centroid_calculation(data, Label,n_clusters)
      if numpy.all(centroid_previous == centroids):
          break
          
  Error = Compute_sse(data, Label, centroids,n_clusters)
  return centroids,Error,Label

In [ ]:
def predict(data):
        distance = Distance_calculation(data, centroid_previous)
        return Nearest_cluster(distance)

In [ ]:
Maximum_iterations = 100
Number_of_Clusters = 3
Centroid, Error, Label = fit(Df, Number_of_Clusters, Maximum_iterations)

(3, 21)


In [ ]:
Centroid.shape

(3, 21)

In [ ]:
Error

12475.588800791782

In [ ]:
Label

array([1, 2, 2, 1, 1, 2, 1, 1, 2, 0, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 0, 2,
       1, 1, 1, 1, 1, 0, 1, 2, 2, 1, 0, 1, 2, 1, 1, 1, 1, 1, 0, 0, 2, 2,
       2, 0, 2, 1, 1, 1, 1, 0, 0, 0, 1, 2, 0, 2, 1, 1, 2, 1, 2, 0, 2, 0,
       2, 0, 0, 0, 1, 1, 2, 0, 1, 1, 0, 1, 1, 1, 2, 0, 2, 0, 1, 2, 1, 1,
       0, 2, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 2, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 2, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 2, 1, 2, 1, 0, 1, 0, 1, 2, 0, 0,
       0, 0, 0, 2, 1, 0, 1, 0, 1, 0, 2, 1, 2, 0, 1, 2, 2, 1, 1, 1, 1, 1,
       1, 2, 1, 1, 0, 1, 1, 2, 2, 2, 0, 0, 2, 2, 1, 1, 0, 1, 1, 2, 2, 1,
       1, 1, 1, 1, 1, 2, 2, 1, 2, 0, 2, 2, 1, 2, 2, 0, 2, 2, 0, 0, 0, 0,
       2, 0, 0, 0, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 2,
       1, 0, 2, 0, 1, 0, 2, 2, 1, 2, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2,
       1, 1, 2, 1, 0, 1, 1, 2, 1, 0, 0, 1, 2, 0, 2,

PAM ALGORITHM

In [ ]:
def Convergance(medoid_prevoius, medoids):
    return set([tuple(x) for x in medoid_prevoius]) == set([tuple(x) for x in medoids])

In [ ]:
def New_medoids(data, medoids,cost,labels,n_clusters):
  for i in set(labels): 
    for j in data[labels == i]:
      old_medoid = copy.deepcopy(medoids[i])
      medoids[i] = j
      distance = Distance_calculation(data, medoids,n_clusters)
      labels = Nearest_cluster(distance)
      new_cost = Compute_sse(data, labels, medoids,n_clusters)
      if new_cost < cost:
        print("cost is ",new_cost)
        cost = new_cost
      else:
        medoids[i] = old_medoid
  return medoids

In [ ]:
def fit(data,n_clusters, max_iter, random_state=420):
  medoids = initializ_centroids(data,random_state,n_clusters)
  i = 0
  converged = False
  while( i<=max_iter and not converged):
      medoid_prevoius = copy.deepcopy(medoids)
      distance = Distance_calculation(data, medoid_prevoius,n_clusters)
      labels = Nearest_cluster(distance)
      cost = Compute_sse(data, labels, medoids,n_clusters)
      print("original cost is ",cost)
      medoids = New_medoids(data,medoids,cost,labels,n_clusters)
      converged = Convergance(medoid_prevoius,medoids)
      i=i+1
  error = Compute_sse(data, labels, medoids,n_clusters)
  print(i)
  return medoids,error,labels

In [ ]:
Maximum_iterations=100
K = 3
Medoids, Error, Label_medoid = fit(Df, K, Maximum_iterations)

original cost is  19932.38319206306
cost is  18152.89032264604
cost is  17736.5541594413
cost is  17542.94886549675
cost is  17394.825461758148
cost is  16668.283219960125
cost is  16660.788360590086
cost is  16565.685468431442
cost is  16397.352497372645
cost is  16393.826530331396
cost is  16078.66885442997
original cost is  16078.66885442997
cost is  15973.553906729669
cost is  15781.00098840552
cost is  15760.741834251628
cost is  15519.789805619206
original cost is  15519.789805619206
cost is  15518.884977846305
cost is  15515.053459927747
original cost is  15515.053459927747
4


In [ ]:
Error

15515.053459927747

In [ ]:
Label_medoid

array([1, 0, 1, 0, 1, 1, 2, 2, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 2, 2, 2, 1,
       0, 2, 0, 0, 0, 2, 0, 1, 1, 2, 2, 0, 1, 0, 0, 0, 2, 2, 2, 2, 1, 1,
       0, 2, 1, 0, 0, 0, 0, 2, 2, 2, 0, 0, 2, 1, 0, 2, 1, 0, 1, 2, 0, 2,
       1, 2, 2, 2, 0, 0, 1, 2, 0, 0, 2, 0, 0, 0, 1, 2, 1, 0, 0, 0, 0, 0,
       2, 0, 2, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 2, 0, 2, 0, 2, 0,
       2, 0, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2,
       2, 2, 2, 2, 1, 2, 0, 1, 1, 2, 2, 1, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 1, 0, 2, 2, 0, 0, 2, 2, 0, 0, 0, 0, 1, 0, 2, 2, 2, 0, 1, 2, 2,
       2, 2, 2, 1, 0, 2, 2, 2, 0, 2, 1, 0, 1, 2, 2, 1, 1, 0, 0, 0, 2, 0,
       0, 0, 2, 0, 2, 0, 0, 1, 0, 1, 2, 2, 0, 1, 0, 0, 2, 0, 0, 1, 0, 2,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 2, 1, 1, 2, 1, 1, 2, 2, 2, 2,
       1, 2, 2, 2, 2, 1, 1, 0, 1, 0, 1, 2, 0, 0, 0, 0, 2, 0, 2, 2, 2, 1,
       0, 1, 1, 2, 0, 2, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,